In [9]:
import pandas as pd 

df = pd.read_csv("data/raw_source/metadata/metadata.csv")

In [10]:
dfClean = df[["fullId"]].apply(lambda x: x.str.split(".")).apply(lambda x: x.str[1])
dfClean

,fullId
0,1004f30be305f33d28a1548e344f0e2e
1,100f39dce7690f59efb94709f30ce0d2
2,101354f9d8dede686f7b08d9de913afe
3,1018f01d42ae7fad52249d8432f6087e
4,1022fe7dd03f6a4d4d5ad9f13ac9f4e7
...,...
12283,1a6a520652aa2244146fa8a09fad6c38
12284,4ab745d5e834afd23765349fd8e5584
12285,b3bbd60c89f54f1231a50841704a69bf
12286,102273fdf8d1b90041fbc1e2da054acb


In [11]:
dfClean[["y"]] = df[["category"]].apply(lambda x: x.str.split(",")).apply(lambda x: x.str[0])

In [12]:
dfClean

,fullId,y
0,1004f30be305f33d28a1548e344f0e2e,WallArt
1,100f39dce7690f59efb94709f30ce0d2,Chair
2,101354f9d8dede686f7b08d9de913afe,Speaker
3,1018f01d42ae7fad52249d8432f6087e,Sword
4,1022fe7dd03f6a4d4d5ad9f13ac9f4e7,Chair
...,...,...
12283,1a6a520652aa2244146fa8a09fad6c38,Lamp
12284,4ab745d5e834afd23765349fd8e5584,CellPhone
12285,b3bbd60c89f54f1231a50841704a69bf,RiceCooker
12286,102273fdf8d1b90041fbc1e2da054acb,Lamp


In [15]:
temp = (df["category"].astype(str) + " " + df["wnlemmas"].astype(str) + " " + df["name"].astype(str) + " " + df["tags"].astype(str)).str.replace("nan", " ").str.replace("None", " ").str.replace(",", " ").str.replace(";", " ").str.replace(":", " ").str.replace(".", " ").str.replace("(", " ").str.replace(")", " ").str.replace("!", " ").str.replace("?", " ").str.replace("'", " ").str.replace('"', " ").str.replace("/", " ").str.replace("\\", " ").str.replace("-", " ").str.replace("_", " ").str.replace("=", " ").str.replace("+", " ").str.replace("*", " ").str.replace("&", " ").str.replace("^", " ").str.replace("%", " ").str.replace("$", " ").str.replace("#", " ").str.replace("@", " ").str.replace(" +", " ").str.lower()
# remove duplicate words
temp = temp.apply(lambda x: " ".join(pd.unique(x.split(" ")))).str.replace("  ", " ")
dfClean[["yFull"]] = pd.DataFrame(temp)

In [18]:
dfClean.to_csv("data/processed/complete.csv")

In [21]:
import os 
toKeep = os.listdir("data/processed/pytorch_graph")
toKeep = [x.split(".")[0] for x in toKeep if x.endswith(".pt")]

In [23]:
dfClean

,fullId,y,yFull
0,1004f30be305f33d28a1548e344f0e2e,WallArt,wallart globe blue spruce
1,100f39dce7690f59efb94709f30ce0d2,Chair,chair recliner reclining lounger couch carpet
2,101354f9d8dede686f7b08d9de913afe,Speaker,speaker attributes loudspeaker unit system sou...
3,1018f01d42ae7fad52249d8432f6087e,Sword,sword blade brand steel weapon
4,1022fe7dd03f6a4d4d5ad9f13ac9f4e7,Chair,chair officechair office fauteuil de bureau
...,...,...,...
12283,1a6a520652aa2244146fa8a09fad6c38,Lamp,lamp walllamp wall
12284,4ab745d5e834afd23765349fd8e5584,CellPhone,cellphone sony ericsson w i mobile
12285,b3bbd60c89f54f1231a50841704a69bf,RiceCooker,ricecooker national
12286,102273fdf8d1b90041fbc1e2da054acb,Lamp,lamp desklamp anglepoise bulb desk clamp floor...


In [30]:
dfToKeep = dfClean[dfClean["fullId"].isin(toKeep)].reset_index(drop=True)
dfToKeep.to_csv("data/processed/completeToKeep.csv")

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")

In [11]:
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
import os
import pandas as pd

df = pd.read_csv("data/processed/completeToKeep.csv")[["yFull"]]

In [12]:
import csv

# Suppose you have a list l
l = df["yFull"].to_list()

# Open the CSV file in write mode
with open('generated.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["id", "generated_text"])

    for i, tags in enumerate(l):
        # Prepare the prompt
        prompt = f"Object and definition is |Object tags: {tags}|Definition:"

        # Tokenize the prompt and move it to the CUDA device
        encod = tokenizer.encode(prompt, return_tensors="pt")
        # Generate the text
        generated = model.generate(encod, max_length=50, do_sample=False, pad_token_id=tokenizer.eos_token_id)

        # Decode the generated text
        text = tokenizer.decode(generated[0])

        # Write the id and the generated text to the CSV file
        writer.writerow([i, text])
        print(text)

Object and definition is |Object tags: wallart globe blue spruce |Definition: A globe is a three-dimensional representation of the Earth, usually mounted on a stand.
The object |Object tags: wallart globe blue spruce 
Object and definition is |Object tags: chair recliner reclining lounger couch carpet|Definition: A chair with a back that reclines to a nearly horizontal position.
The chair recliner is a chair with a back that reclines to
Object and definition is |Object tags: speaker attributes loudspeaker unit system sound audio music|Definition: A loudspeaker is a device that converts electrical signals into sound waves.
The speaker is a device that converts electrical signals into sound waves.



KeyboardInterrupt: 